In [37]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [38]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle"

In [39]:
%cd /content/gdrive/My Drive/kaggle

/content/gdrive/My Drive/kaggle


In [40]:
!kaggle competitions download -c dog-breed-identification

 99% 685M/691M [00:05<00:00, 127MB/s]
100% 691M/691M [00:05<00:00, 124MB/s]


In [41]:
!unzip \*.zip  && rm *.zip

Archive:  dog-breed-identification.zip
replace labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [45]:
import torch
from PIL import Image
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from skimage import io
import pandas as pd
from torch.utils.data import DataLoader
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
from torchvision.datasets import ImageFolder

labels = pd.read_csv('https://drive.google.com/file/d/1-2PurAI18uOOLFfo9oySYT0VVfR9AFc4/view?usp=sharing')
labelnames = pd.read_csv('https://drive.google.com/file/d/1-1ZxXjbu9zZgHu1_V2UDqGO1YzOvjN46/view?usp=sharing').keys()[1:]
#print("Train folder has ", len(os.listdir(PATH+'train')),'images which matches with label\'s', len(labels),'images')

codes = range(len(labelnames))
breed_to_code = dict(zip(labelnames, codes))
code_to_breed = dict(zip(codes, labelnames))
labels['target'] =  [breed_to_code[x] for x in labels.breed]
labels['rank'] = labels.groupby('breed').rank()['id']
labels_pivot = labels.pivot('id', 'breed', 'target').reset_index().fillna(0)

train = labels_pivot.sample(frac=0.85)
test = labels_pivot[~labels_pivot['id'].isin(train['id'])]


transform = transforms.Compose([transforms.Resize((224,224), interpolation=torchvision.transforms.InterpolationMode.BICUBIC),
                                transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5 ))])

class DogBreedDataset(Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, img_dir, label, transform):
        'Initialization'
        self.img_dir = img_dir
        self.transform = transform
        self.label = label

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.label)

    def __getitem__(self, index):
        if self.label is not None:
            img_name = '{}.jpg'.format(self.label.iloc[index, 0])
            fullname = self.img_dir + img_name
            image = Image.open(fullname)
            label = self.label.iloc[index, 1:].astype('float').to_numpy()
            label = np.argmax(label)
            if self.transform:
                image = self.transform(image)
            return [image, label]


train_img = DogBreedDataset('https://drive.google.com/drive/folders/1G_en_EynMGODZKVR4YSbgiO3jWNIzdtF?usp=sharing', train, transform = transform)
test_img = DogBreedDataset('https://drive.google.com/drive/folders/1G_en_EynMGODZKVR4YSbgiO3jWNIzdtF?usp=sharing', test, transform = transform)

trainloader = torch.utils.data.DataLoader(train_img,
                                            batch_size = 128,
                                            shuffle = True,
                                            num_workers = 0)

testloader = torch.utils.data.DataLoader(test_img,
                                           batch_size = 128,
                                           shuffle = False,
                                           num_workers = 0)

dataiter = iter(trainloader)

images, labels = dataiter.next()

if torch.cuda.is_available():
  device = 'cuda' 
else:
  device = 'cpu'


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
    
        self.pool = nn.MaxPool2d(2, 2)
    
        self.fc1 = nn.Linear(128 * 28 * 28,500 )
        self.fc2 = nn.Linear(500, 120)
    
        self.dropout = nn.Dropout(0.3)
        
        
    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.view(-1, 128 * 28 * 28)
    
        x = self.dropout(x)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# instantiate the CNN
net = Net()

# move tensors to GPU if CUDA is available
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

epochs = 10

epoch_log = []
loss_log = []
accuracy_log = []

# Iterate for a specified number of epochs
for epoch in range(epochs):  
    print(f'Starting Epoch: {epoch+1}...')

    running_loss = 0.0
    # Each cycle is a minibatch
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # Move our data to GPU
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward -> backprop + optimize
        outputs = net(inputs) 
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step()

        # Print Training statistics - Epoch/Iterations/Loss/Accuracy
        running_loss += loss.item()
        if i % 50 == 49:    
            correct = 0 
            total = 0 

            with torch.no_grad():
                # Iterate through the testloader iterator
                for data in testloader:
                    images, labels = data
                    # Move our data to GPU
                    images = images.to(device)
                    labels = labels.to(device)
                    
                    # Foward propagate our test data batch through our model
                    outputs = net(images)

                    _, predicted = torch.max(outputs.data, dim = 1)
                    # Keep adding the label size or length to the total variable
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                accuracy = 100 * correct / total
                epoch_num = epoch + 1
                actual_loss = running_loss / 50
                print(f'Epoch: {epoch_num}, Mini-Batches Completed: {(i+1)}, Loss: {actual_loss:.3f}, Test Accuracy = {accuracy:.3f}%')
                running_loss = 0.0

    # Store training stats after each epoch
    epoch_log.append(epoch_num)
    loss_log.append(actual_loss)
    accuracy_log.append(accuracy)

print('Finished Training')

AttributeError: ignored